In [1]:
import tensorflow as tf
from keras.layers import Dense
import json
import numpy as np
import librosa
import pandas as pd
from model_utils import save_model

In [2]:
from tensorflow import keras

In [3]:
import glob
import soundfile

In [47]:
y, sr = librosa.load('/Users/cooky/Desktop/data/02-03.wav')
y

array([ 6.5707300e-06,  3.4268778e-06, -1.4703763e-06, ...,
        1.3547251e-06,  1.1079610e-06,  9.1189868e-06], dtype=float32)

In [4]:
X_input = []
x_label = []
y_output = []

for i in range(68):
    x, sr_x = librosa.load('/Users/cooky/Desktop/data/'+str(i).zfill(2)+'.wav', mono=True)
    # x = np.transpose(x)[0]
    chg = glob.glob('/Users/cooky/Desktop/data/'+str(i).zfill(2)+'-*.wav')[0]
    y, sr_y = librosa.load(chg, mono=True)
    # y = np.transpose(y)[0]
    if (len(y) != len(x)) or (sr_x != sr_y):
        print('Err ' + str(i))
    else:
        print(i)
        print(x.max())
        print(x.min())
        X_input.extend(x[:1000])
        y_output.extend(y[:1000])
        for _ in range(1000):
            x_label.append(int(chg[-6:-4]))

0
0.68489426
-0.6581681
1
0.65787685
-0.63718987
2
1.0353678
-1.0510951
3
1.0874541
-1.0620916
4
0.9818718
-0.9783458
5
1.0505552
-0.98955077
6
0.98871905
-0.9707296
7
1.0080343
-0.9800307
8
1.0094304
-1.0193173
9
1.016585
-0.99791545
10
0.37009746
-0.4280415
11
0.44889787
-0.52701026
12
0.31235188
-0.50525594
13
0.51289785
-0.50687844
14
1.044419
-0.8235012
15
1.001747
-0.9962708
16
1.0246911
-1.048015
17
0.73426086
-0.7601719
18
1.0061857
-0.9301044
19
0.9574974
-1.0005014
20
1.0237501
-1.0219729
21
0.82687724
-0.75175816
22
0.8284106
-0.79243577
23
0.72521424
-0.65278095
24
0.6610688
-0.6092189
25
0.8150219
-0.77616274
26
0.30358335
-0.28720614
27
0.80046046
-0.85944706
28
0.7441885
-0.83397543
29
0.7010561
-0.6755326
30
0.78778726
-0.62524736
31
0.8573344
-0.91760314
32
0.93862724
-0.91508114
33
0.8137678
-0.8642007
34
0.5354991
-0.5389247
35
0.6721084
-0.7246859
36
0.40343222
-0.3587649
37
0.37280142
-0.40261877
38
0.7427456
-0.7412466
39
0.28842014
-0.27153316
40
0.43958533
-0.33

In [5]:
X_input = np.array(X_input)
y_output = np.array(y_output)
x_label = np.array(x_label)

In [7]:
X_inputt = np.vstack((X_input,x_label)).T
# X_inputt = X_input.reshape((X_input.shape[0],1))
y_outputt = y_output.reshape((y_output.shape[0],1))

In [8]:
X_inputt[1]

array([7.44158251e-06, 1.00000000e+00])

In [9]:
y_outputt[1]

array([-1.460835e-07], dtype=float32)

In [17]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(None, 2)))
model.add(keras.layers.Reshape(target_shape=(2, 1)))
model.add(keras.layers.Dense(4, activation='relu', kernel_initializer='random_normal', bias_initializer='random_normal'))
model.add(keras.layers.Dense(8, activation='relu', kernel_initializer='random_normal', bias_initializer='random_normal'))
model.add(keras.layers.Conv1D(4, 3, dilation_rate=2,padding='same', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='random_normal'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, kernel_initializer='orthogonal', bias_initializer='random_normal', activation='tanh'))

In [18]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001), loss='mse', metrics=['mse','mae'])

In [19]:
with tf.device('/GPU:0'):
    model.fit(x=X_inputt, y=y_outputt, validation_split=0.1, batch_size=16, epochs=20)

Epoch 1/20
3825/3825 [==============================] - 2s 464us/step - loss: 0.0077 - mse: 0.0077 - mae: 0.0350 - val_loss: 0.0302 - val_mse: 0.0302 - val_mae: 0.0705
Epoch 2/20
3825/3825 [==============================] - 2s 438us/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0347 - val_loss: 0.0298 - val_mse: 0.0298 - val_mae: 0.0606
Epoch 3/20
3825/3825 [==============================] - 2s 442us/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0350 - val_loss: 0.0298 - val_mse: 0.0298 - val_mae: 0.0635
Epoch 4/20
3825/3825 [==============================] - 2s 448us/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0349 - val_loss: 0.0297 - val_mse: 0.0297 - val_mae: 0.0618
Epoch 5/20
3825/3825 [==============================] - 2s 450us/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0347 - val_loss: 0.0297 - val_mse: 0.0297 - val_mae: 0.0608
Epoch 6/20
3825/3825 [==============================] - 2s 438us/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0347 - val_loss: 0.0299 - val_mse: 0.0299 - val_mae:

KeyboardInterrupt: 

In [15]:
X_input.shape[0]

829951